In [9]:
Mallikarjun edara
#import required libraries
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #layers in neural network
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re  #regular expression operations

from sklearn.preprocessing import LabelEncoder

In [11]:
#Loading the sentiment dataset
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]
type(data)


pandas.core.frame.DataFrame

In [12]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))#only keeping a-z,A-Z,0-9 in the data

In [13]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')#removing retweets

In [14]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')#tokenizing the sentence
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)#padding the feature matrix - add zeros for matching length

In [15]:
X.shape

(13871, 28)

In [16]:
def createmodel():
    embed_dim=128 #dimension of the embedded layer
    lstm_out=196 #LSTM layer neurons
    model = Sequential()#Sequential neural network
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
    #print(model.summary)

In [17]:
labelencoder = LabelEncoder()#conversion of categorical to Numerical
#fitting the model
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [18]:
print(data['sentiment'])
print(integer_encoded)

0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object
[1 2 1 ... 2 0 2]


In [19]:
# train model
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)#more messages for higher verbose
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)#model metrics

291/291 - 32s - loss: 0.8266 - accuracy: 0.6459
144/144 - 2s - loss: 0.7956 - accuracy: 0.6549
0.7955935597419739
0.6548711061477661
['loss', 'accuracy']


#1.Save the model and use the saved model to predict on new text data


In [20]:
model.save("model.h5")#saving the model

In [21]:
from keras.models import load_model #importing the package to get the saved model
model_save= load_model("model.h5")  #loading the model which is saved

In [28]:
import numpy as np
#processing the input text
tweet = ['A lot of good things are happening.We are respected again throughout the world, and that is a great thing']
tweet = tweet[0].lower()
tweet = re.sub('[^a-zA-z0-9\s]','',tweet)
tweet = tokenizer.texts_to_sequences(tweet)#tokenizing the sentence
tweet = pad_sequences(tweet, maxlen=28, dtype='int32', value=0)#padding - add zeros to match the sentence length
#Standard analyzer defines up to three basic polar emotions (positions, negative, neutral)
sentiment = model_save.predict_classes(tweet,batch_size=1, verbose=2)[0] #predicting the sentence text
if sentiment == 0:
    print("negative")
elif sentiment==1:
    print("neutral")
else:
    print("positive")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


103/103 - 1s
positive


#2.Applying gridsearchCV on the source code

In [29]:
#importing the required libraries
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
# applying GridSearchCV on model
model = KerasClassifier(build_fn=createmodel,verbose=2)#applying multiple hyper parameters for model initiation
#hyper parameters
batch_size= [40, 50, 60]
epochs = [1, 2, 3]
param_grid= dict(batch_size=batch_size, epochs=epochs) #creating dictionary for batch size and no of epochs
grid = GridSearchCV(estimator=model, param_grid=param_grid) #applying dictionary with hyper parameters for GridSearchCV
grid_result=grid.fit(X_train,y = Y_train)#fitting the model
#summary
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

186/186 - 23s - loss: 0.8445 - accuracy: 0.6367
47/47 - 1s - loss: 0.7590 - accuracy: 0.6530
186/186 - 22s - loss: 0.8467 - accuracy: 0.6367
47/47 - 1s - loss: 0.7730 - accuracy: 0.6659
186/186 - 23s - loss: 0.8493 - accuracy: 0.6322
47/47 - 1s - loss: 0.8223 - accuracy: 0.6611
186/186 - 23s - loss: 0.8425 - accuracy: 0.6334
47/47 - 1s - loss: 0.8151 - accuracy: 0.6663
186/186 - 22s - loss: 0.8438 - accuracy: 0.6403
47/47 - 1s - loss: 0.7933 - accuracy: 0.6738
Epoch 1/2
186/186 - 23s - loss: 0.8475 - accuracy: 0.6349
Epoch 2/2
186/186 - 20s - loss: 0.6969 - accuracy: 0.7043
47/47 - 1s - loss: 0.7336 - accuracy: 0.6799
Epoch 1/2
186/186 - 22s - loss: 0.8415 - accuracy: 0.6400
Epoch 2/2
186/186 - 21s - loss: 0.7024 - accuracy: 0.7030
47/47 - 1s - loss: 0.7296 - accuracy: 0.6832
Epoch 1/2
186/186 - 23s - loss: 0.8485 - accuracy: 0.6365
Epoch 2/2
186/186 - 20s - loss: 0.6855 - accuracy: 0.7061
47/47 - 1s - loss: 0.7483 - accuracy: 0.6880
Epoch 1/2
186/186 - 22s - loss: 0.8566 - accuracy: 0